# Welcome to introduction to dashboards with Plotly and Dash

-------------------------------------------------------------------------------------------------------------------------------


### Workshop facilitators: Laura Gutierrez Funderburk, Hanh Tong

### About this workshop

In this workshop we will explore key characteristics of the housing market in Vancouver, BC. 

It is important to note that this workshop assumes:

1. Data cleaning and exploration was completed prior to developing the dashboard
2. Some comfort with `pandas` and visualization is assumed
3. Comfort navigating the Jupyter environment is needed



### Workshop schedule:
-------------------------------------------------------------------------------------------------------------------------------


#### 1. Part I: Data exploration

In this section, we will first spend time getting familiar with the data. We will use the `pandas` and `plotly` libraries, we will also explore the `DEX` feature within Noteable to ease getting a good sense for what the data contains.

In this section, we will also explore the notion of factoring code into functions, and the notion of writing a Python script that we can use to easily recreate our results. 

#### 2. Part II: Dashboard components

In this section, we will take what we built together in part I and explore the main components in a Dash dashboard. 

## Part II: Dashboard components
-------------------------------------------------------------------------------------------------------------------------------


The four main components in a Dash dashboard are:

1. A .py script (also known as your dashboard app) `app.py`

2. A requirements.txt file with the Python dependencies for your app

3. A Procfile

4. A .gitignore file


### The .py script `app.py`

-------------------------------------------------------------------------------------------------------------------------------


The anatomy of a .py script is as follows:

```python
import math

def function1(par1: int, par2: str) -> None:
    """
    Parameters
    ----------
        par1: (dataframe object) reshaped data frame object with mortage, delinquency and population data
        par2: (string) "box", "violin", "scatter", "line"
         
    Returns:
    --------
        None
    """
    print(f"{par1} is spelled '{par2}'")
        
if __name__ == '__main__':  
    
    # Code that does something
    function1(1, "one")
```

### A running example

-------------------------------------------------------------------------------------------------------------------------------

Let's package the content of the function `graph_region` that we crafted in part I. The script looks as follows:

In [ ]:
%load ./nb-scripts/base_script.py

In [ ]:
%run -i ./nb-scripts/base_script.py


### What is wrong with the plots being laid out this way?
-------------------------------------------------------------------------------------------------------------------------------


A lot...

This is hard to read, hard to interact with, and overwhelming. We introduce the app machinery.

The `app.py` script is just a .py script with additional machinery. 


### The `app.py` additional machinery
-------------------------------------------------------------------------------------------------------------------------------

Our app is going to contain mostly the same as a regular .py script, plus a few new items:

- Dash library imports
- External stylesheets
- App initialization
- App layout
- Decorator callbacks

### Dash library imports
-------------------------------------------------------------------------------------------------------------------------------

```python

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

```

### External stylesheets
-------------------------------------------------------------------------------------------------------------------------------

```python

# Stylesheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


```

Dash applications are rendered in the web browser with CSS and JavaScript. On page load, Dash serves a small HTML template that includes references to the CSS and JavaScript that are required to render the application. You can use stylesheets in your dashboard, similarly to how you'd use them to style your website.

Read more https://dash.plotly.com/external-resources

### App initialization
-------------------------------------------------------------------------------------------------------------------------------

```python

# Intialize app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server


```


### App layout
-------------------------------------------------------------------------------------------------------------------------------

```python

app.layout = html.Div([sidebar, content])


```

This component describes what the application looks like.

https://dash.plotly.com/layout

### Decorator callbacks
-------------------------------------------------------------------------------------------------------------------------------

```python

@app.callback(
     Output('graph-id', 'figure'),
    Input('value-id', 'value')
def update_figure(parameter):
    """
    Parameters
    ----------
        parameter: This is the value that will be updated in the app
    """
    
    # Code that uses the variable value to make a change 
    ...
    # Code that generates figure based on value
    fig = ...

    return fig


```


Decorator callbacks act Python functions that are automatically called by Dash whenever an input component's property changes. This is what makes it possible for dashboards to be interactive.

Read more https://dash.plotly.com/basic-callbacks


```python
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# ----------------------------------------------------------------------------------#
# Read data


# ----------------------------------------------------------------------------------#
# App section

# Stylesheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Intialize app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

app.layout = # some code with our layout


@app.callback(
     Output('graph-id', 'figure'),
    Input('value-id', 'value')
def update_figure(parameter):
    """
    Parameters
    ----------
        parameter: This is the value that will be updated in the app
    """
    
    # Code that uses the variable value to make a change 
    ...
    # Code that generates figure based on value
    fig = ...

    return fig


if __name__ == '__main__':  
    app.run_server(debug=True) 
```



-------------------------------------------------------------------------------------------------------------------------------
